In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import json
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
with open('sarcasm.json','r') as f:
    datastore = json.load(f)
    
sentences = []
labels = []

for item in datastore:
    labels.append(item['is_sarcastic'])
    sentences.append(item['headline']) 

In [6]:
train_size = 20000
vocab_size = 1000
max_length = 120
embedding_dims = 16
oov_token = '<oov>'
padding = 'post'
trunc_type = 'post'

In [7]:
train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

test_sentences = sentences[train_size:]
test_labels = labels[:train_size]

In [8]:
token = Tokenizer(num_words = vocab_size, oov_token = oov_token)
token.fit_on_texts(train_sentences)
word_index = token.word_index
train_seq = token.texts_to_sequences(train_sentences)
train_pad = pad_sequences(train_seq, padding = padding, truncating = trunc_type, maxlen = max_length)

In [9]:
test_seq = token.texts_to_sequences(test_sentences)
test_pad = pad_sequences(test_seq, padding = padding, truncating = trunc_type, maxlen = max_length)

train_pad = np.array(train_pad)
test_pad = np.array(test_pad)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dims, input_length = max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8, return_sequences = True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           16000     
_________________________________________________________________
bidirectional (Bidirectional (None, 120, 16)           1600      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32)                4224      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 22,369
Trainable params: 22,369
Non-trainable params: 0
_________________________________________________________________


In [11]:
num_epochs = 10
history = model.fit(train_pad, epochs = num_epochs, validation_data = test_pad)

Epoch 1/10


ValueError: in user code:

    /home/soulreaper/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/soulreaper/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/soulreaper/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/soulreaper/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/soulreaper/anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:541 train_step  **
        self.trainable_variables)
   